## Orlando Crime Data
* Read the crime data for Seattle from https://data.cityoforlando.net/Orlando-Police/OPD-Crimes/4y9m-jbmz/data
* Take the data from year 2015-present
* Formatted the data with Type of crime, Date Occurred and crime count
* Store the formatted data in csv file

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

In [2]:
# File to Load
csv_path = "../../Data_folder/OPD_Crime_data_2015.csv"
data_file = pd.read_csv(csv_path)
data_file.head()

,Case Number,Case Date Time,Case Location,Case Offense Location Type,Case Offense Category,Case Offense Type,Case Offense Charge Type,Case Disposition,Status,Location
0,2015-00000019,01/01/2015 12:18:00 AM,4200 Block of L B MCLEOD RD,Apartment/Condo,Theft,Theft from motor vehicle except parts/accessories,Committed,Closed,Mapped,"(28.50874428, -81.42880573)"
1,2015-00000033,01/01/2015 12:33:00 AM,1000 Block of CITRUS ST,Residence/Other,Theft,All other larceny,Committed,Closed,Mapped,"(28.52892411, -81.39402909)"
2,2015-00000040,01/01/2015 12:36:00 AM,1 Block of W CHURCH ST,Bar/Night Club/Country Club,Assault,Simple assault,Committed,Closed,Mapped,"(28.54032149, -81.37988988)"
3,2015-00000063,01/01/2015 12:51:00 AM,6000 Block of UNIVERSAL BV,Theme Park,Robbery,Robbery,Committed,Closed,Mapped,"(28.47343236, -81.4641098)"
4,2015-00000069,01/01/2015 12:56:00 AM,4900 Block of CASON COVE DR,Apartment/Condo,Burglary,Burglary/breaking and entering,Attempted,Closed,Mapped,"(28.49922815, -81.44417626)"


In [3]:
len(data_file)

64743

In [8]:
data_file['Case Date Time']= data_file['Case Date Time'].astype('datetime64[ns]')
data_file['Case Date'] = [d.date() for d in data_file['Case Date Time']]
data_file['Case Time'] = [d.time() for d in data_file['Case Date Time']]
data_file['Case Date']= data_file['Case Date'].astype('datetime64[ns]')

In [9]:
data_file.dtypes

Case Number                           object
Case Date Time                datetime64[ns]
Case Location                         object
Case Offense Location Type            object
Case Offense Category                 object
Case Offense Type                     object
Case Offense Charge Type              object
Case Disposition                      object
Status                                object
Location                              object
Case Date                     datetime64[ns]
Case Time                             object
dtype: object

In [11]:
#Read all the categories of crime 
cats = data_file["Case Offense Category"].unique()
cats

array(['Theft', 'Assault', 'Robbery', 'Burglary', 'Vehicle Theft',
       'Fraud', 'Narcotics', 'Arson', 'Homicide', 'Embezzlement',
       'Kidnapping'], dtype=object)

In [12]:
cats_df = pd.DataFrame({"Case Offense Category":cats})
cats_df.to_csv('categories.csv')

#Go to the excel and edit the file

In [13]:
#read the new csv file with final category colummn and add to our new dataframe

cats_key = pd.read_csv('category_keys.csv')
cats_key.head()

,Case Offense Category,Category Final
0,Theft,LARCENY-THEFT
1,Assault,AGGRAVATED ASSAULT
2,Robbery,LARCENY-THEFT
3,Burglary,BURGLARY
4,Vehicle Theft,MOTOR VEHICLE THEFT


In [14]:
orlando_data = pd.merge(data_file,cats_key,on="Case Offense Category",how="left")
orlando_data.head()

,Case Number,Case Date Time,Case Location,Case Offense Location Type,Case Offense Category,Case Offense Type,Case Offense Charge Type,Case Disposition,Status,Location,Case Date,Case Time,Category Final
0,2015-00000019,2015-01-01 00:18:00,4200 Block of L B MCLEOD RD,Apartment/Condo,Theft,Theft from motor vehicle except parts/accessories,Committed,Closed,Mapped,"(28.50874428, -81.42880573)",2015-01-01,00:18:00,LARCENY-THEFT
1,2015-00000033,2015-01-01 00:33:00,1000 Block of CITRUS ST,Residence/Other,Theft,All other larceny,Committed,Closed,Mapped,"(28.52892411, -81.39402909)",2015-01-01,00:33:00,LARCENY-THEFT
2,2015-00000040,2015-01-01 00:36:00,1 Block of W CHURCH ST,Bar/Night Club/Country Club,Assault,Simple assault,Committed,Closed,Mapped,"(28.54032149, -81.37988988)",2015-01-01,00:36:00,AGGRAVATED ASSAULT
3,2015-00000063,2015-01-01 00:51:00,6000 Block of UNIVERSAL BV,Theme Park,Robbery,Robbery,Committed,Closed,Mapped,"(28.47343236, -81.4641098)",2015-01-01,00:51:00,LARCENY-THEFT
4,2015-00000069,2015-01-01 00:56:00,4900 Block of CASON COVE DR,Apartment/Condo,Burglary,Burglary/breaking and entering,Attempted,Closed,Mapped,"(28.49922815, -81.44417626)",2015-01-01,00:56:00,BURGLARY


In [15]:
crimes = orlando_data["Category Final"].unique()
crimes

array(['LARCENY-THEFT', 'AGGRAVATED ASSAULT', 'BURGLARY',
       'MOTOR VEHICLE THEFT', 'FRAUD', 'DRUG OFFENSE', 'ARSON',
       'MURDER AND NON-NEGLIGENT MANSLAUGHTER', 'MISC'], dtype=object)

In [16]:
# Add columns for each crime category, 0 = crime is not this category, 1 = crime is this category
for c in crimes:
    orlando_data[c]= orlando_data['Category Final'].apply(lambda s: int(s==c))
# Group by date and export the counts to csv
orlando_final_df = orlando_data.groupby(['Case Date'])
orlando_final_df[crimes].sum().to_csv('orlando_by_date.csv') 
orlando_final_df[crimes].sum()

,LARCENY-THEFT,AGGRAVATED ASSAULT,BURGLARY,MOTOR VEHICLE THEFT,FRAUD,DRUG OFFENSE,ARSON,MURDER AND NON-NEGLIGENT MANSLAUGHTER,MISC
Case Date,,,,,,,,,
2015-01-01,35,20,6,4,2,2,0,0,0
2015-01-02,33,9,6,3,4,9,0,0,0
2015-01-03,38,9,8,2,0,8,0,0,0
2015-01-04,33,10,9,1,3,6,0,0,0
2015-01-05,47,8,8,5,6,4,0,0,0
2015-01-06,47,6,8,1,7,7,0,0,0
2015-01-07,29,4,10,1,10,9,1,0,0
2015-01-08,37,10,11,0,9,10,0,0,0
2015-01-09,28,4,8,0,9,6,0,0,0
